# Movie Recommender

Movie Tinder, where it registers your mood.
Shuffling of Movies
Movies of different Genre

Can consider Decision Trees

Group by Users Id and Movie Id

Grou Presentation Flow:
Make up an issue/story (1 min)
Technical Description of Code (3 min)
How we use the code and how it addresses our issue (4 min) 
Link back (2 min)

In [1]:
# We will first import our basic modules and packages
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
sb.set()

In [2]:
# Let us import the datasets, we will be using the smaller datasets first
movies = pd.read_csv('ratings_small.csv').drop(columns = ['timestamp'])
link = pd.read_csv('links_small.csv').drop(columns= ['tmdbId'])
keywords = pd.read_csv('keywords.csv')
metadata = pd.read_csv('movies_metadata.csv').drop(columns=['vote_count', 'vote_average','video','tagline','status'])
credit = pd.read_csv('credits.csv')

print("Movies dataset is", movies.shape)
print("Links dataset is", link.shape)
print("Keywords dataset is", keywords.shape)
print("Metadata dataset is", metadata.shape)
print("Credits dataset is", credit.shape)

C:\Users\yihao\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Movies dataset is (100004, 3)
Links dataset is (9125, 2)
Keywords dataset is (46419, 2)
Metadata dataset is (45466, 19)
Credits dataset is (45476, 3)


# Data Cleaning

In [3]:
# Movies and Link form a data
data = pd.merge(movies, link, on='movieId')

metadata['imdb_id'] = metadata['imdb_id'].str[2:]
metadata.head()

# Converting metadata IMDB ID column into an Integer
metadata_imdb = pd.DataFrame(metadata['imdb_id'].dropna()).replace('',0).astype(np.int64)

# Rename Meta Data IMDB ID to imdbid
metadata = metadata[metadata.imdb_id.notnull()]
metadata['imdb_id'] = metadata_imdb
metadata = metadata.rename(columns={"imdb_id": "imdbId"})

# Print the results
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdbId,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II


In [13]:
credit.dtypes


cast    object
crew    object
id       int64
dtype: object

In [14]:
data.head()

,userId,movieId,rating,imdbId
0,1,31,2.5,112792
1,7,31,3.0,112792
2,31,31,4.0,112792
3,32,31,4.0,112792
4,36,31,3.0,112792


In [44]:
# Let us try to find the average rating of each movie
average_rating = data.groupby('movieId')['rating'].mean().sort_values(ascending = True)
ratings = pd.merge(link, average_rating, on = 'movieId')
ratings.shape

(9066, 3)

In [32]:
# Let us merge the metadata with the ratings dataset (containing the found average ratings) we just created
merged_data = pd.merge(metadata,ratings, on='imdbId').drop(columns=['adult','poster_path']).drop(columns=['belongs_to_collection','original_title'])
merged_data['id'] = merged_data['id'].astype(np.int64)

# Let us merge the already merged data with the credits dataset
merged_data = pd.merge(merged_data, credit, on = 'id')
display(merged_data.head(3))
print("Final Dataset Size", merged_data.shape)

,budget,genres,homepage,id,imdbId,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,movieId,rating,cast,crew
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1,3.872470,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,113497,en,When siblings Judy and Peter discover an encha...,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,2,3.401869,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,113228,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,3,3.161017,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


Final Dataset Size (9031, 19)


We now have our finalised **Dataset** formed from dropping irrelevant columns and merging multiple different dataset. Well it seems we have 9031 rows to work with. 

But first things first, we need to do some tidying up of the names of the columns and extracting some contents in the columns out of their respective lists


In [2]:
# lets trying cleaning up the cast name with a function that takes in a dataset, we have chosen the first 5 actors/actresses only
def get_cast_name(dataset):
    name = ""
    if len(ast.literal_eval(dataset)) > 5:
        a = 5
    else:
        a = len(ast.literal_eval(dataset))
    for i in range(a):
     #   name += ast.literal_eval(dataset[0])[i]['character']
     #   name += ", "
        name += ast.literal_eval(dataset)[i]['name']
        name += ", "
    return name

# Replacing the Cast Column with our newly found cast
a = merged_data.copy()
cast_list = []
for i in range(len(merged_data)):
    cast_list.append(get_cast_name(a['cast'][i]))

merged_data['cast'] = cast_list

NameError: name 'merged_data' is not defined

In [54]:
# Renaming more columns
merged_data.rename(columns={"original_language": "Language", "original_title":"Title"})
merged_data['genres'][0]


[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [61]:
# Renaming the genre list
def get_genre(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

genre_list = []
for i in range(len(merged_data)):
    genre_list.append(get_genre(merged_data['genres'][i]))
    
merged_data['genres'] = genre_list
merged_data.head()

,budget,genres,homepage,id,imdbId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,movieId,rating,cast,crew
0,30000000,"Animation, Comedy, Family,",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1,3.872470,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"Adventure, Fantasy, Family,",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,2,3.401869,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"Romance, Comedy,",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,3,3.161017,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,16000000,"Comedy, Drama, Romance,",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,4,2.384615,"Whitney Houston, Angela Bassett, Loretta Devin...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,"Comedy,",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5,3.267857,"Steve Martin, Diane Keaton, Martin Short, Kimb...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [62]:
# Renaming the country
def get_country(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

genre_list = []
for i in range(len(merged_data)):
    genre_list.append(get_country(merged_data['production_countries'][i]))
    
merged_data['production_countries'] = genre_list
merged_data.head()

,budget,genres,homepage,id,imdbId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,movieId,rating,cast,crew
0,30000000,"Animation, Comedy, Family,",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]","United States of America,",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1,3.872470,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"Adventure, Fantasy, Family,",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","United States of America,",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,2,3.401869,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"Romance, Comedy,",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","United States of America,",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,3,3.161017,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,16000000,"Comedy, Drama, Romance,",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,[{'name': 'Twentieth Century Fox Film Corporat...,"United States of America,",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,4,2.384615,"Whitney Houston, Angela Bassett, Loretta Devin...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,"Comedy,",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,"[{'name': 'Sandollar Productions', 'id': 5842}...","United States of America,",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5,3.267857,"Steve Martin, Diane Keaton, Martin Short, Kimb...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [63]:
# Renaming the companies
def get_companies(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

genre_list = []
for i in range(len(merged_data)):
    genre_list.append(get_country(merged_data['production_companies'][i]))
    
merged_data['production_companies'] = genre_list
merged_data.head()

,budget,genres,homepage,id,imdbId,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,movieId,rating,cast,crew
0,30000000,"Animation, Comedy, Family,",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,"Pixar Animation Studios,","United States of America,",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1,3.872470,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"Adventure, Fantasy, Family,",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,"TriStar Pictures, Teitler Film, Interscope Com...","United States of America,",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,2,3.401869,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"Romance, Comedy,",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate,","United States of America,",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,3,3.161017,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,16000000,"Comedy, Drama, Romance,",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,"Twentieth Century Fox Film Corporation,","United States of America,",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,4,2.384615,"Whitney Houston, Angela Bassett, Loretta Devin...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,"Comedy,",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,"Sandollar Productions, Touchstone Pictures,","United States of America,",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5,3.267857,"Steve Martin, Diane Keaton, Martin Short, Kimb...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [72]:
# Renaming the Language Column
def get_language(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

genre_list = []
for i in range(len(merged_data)):
    genre_list.append(get_language(merged_data['spoken_languages'][i]))
    
merged_data['spoken_languages'] = genre_list
merged_data.head()

,budget,genres,homepage,id,imdbId,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,title,movieId,rating,cast,crew,release_data
0,30000000,"Animation, Comedy, Family,",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,"Pixar Animation Studios,",...,1995-10-30,373554033.0,81.0,"English,",Toy Story,1,3.872470,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","Led by Woody, Andy's toys live happily in his ..."
1,65000000,"Adventure, Fantasy, Family,",None,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,"TriStar Pictures, Teitler Film, Interscope Com...",...,1995-12-15,262797249.0,104.0,"English, Français,",Jumanji,2,3.401869,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",When siblings Judy and Peter discover an encha...
2,0,"Romance, Comedy,",None,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate,",...,1995-12-22,0.0,101.0,"English,",Grumpier Old Men,3,3.161017,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",A family wedding reignites the ancient feud be...
3,16000000,"Comedy, Drama, Romance,",None,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,"Twentieth Century Fox Film Corporation,",...,1995-12-22,81452156.0,127.0,"English,",Waiting to Exhale,4,2.384615,"Whitney Houston, Angela Bassett, Loretta Devin...","[{'credit_id': '52fe44779251416c91011acb', 'de...","Cheated on, mistreated and stepped on, the wom..."
4,0,"Comedy,",None,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,"Sandollar Productions, Touchstone Pictures,",...,1995-02-10,76578911.0,106.0,"English,",Father of the Bride Part II,5,3.267857,"Steve Martin, Diane Keaton, Martin Short, Kimb...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Just when George Banks has recovered from his ...


In [71]:
# Filling Nan values of the columns
merged_data['homepage'] = merged_data['homepage'].fillna('None')
merged_data['overview'] = merged_data['overview'].fillna('None')
merged_data['release_date'] = merged_data['release_date'].fillna('None')
merged_data.isnull().any()

budget                  False
genres                  False
homepage                False
id                      False
imdbId                  False
original_language       False
original_title          False
overview                False
popularity              False
production_companies    False
production_countries    False
release_date            False
revenue                 False
runtime                 False
spoken_languages        False
title                   False
movieId                 False
rating                  False
cast                    False
crew                    False
release_data            False
dtype: bool

The code below is a one line code of everything above ^, it will process and return the dataset. Note: May take a while to run !!!
# One Line Code to process the Dataset 

In [3]:
movies = pd.read_csv('ratings_small.csv').drop(columns = ['timestamp'])
link = pd.read_csv('links_small.csv').drop(columns= ['tmdbId'])
keywords = pd.read_csv('keywords.csv')
metadata = pd.read_csv('movies_metadata.csv').drop(columns=['vote_count', 'vote_average','video','tagline','status'])
credit = pd.read_csv('credits.csv')

print("Movies dataset is", movies.shape)
print("Links dataset is", link.shape)
print("Keywords dataset is", keywords.shape)
print("Metadata dataset is", metadata.shape)
print("Credits dataset is", credit.shape)

data = pd.merge(movies, link, on='movieId')

metadata['imdb_id'] = metadata['imdb_id'].str[2:]

# Converting metadata IMDB ID column into an Integer
metadata_imdb = pd.DataFrame(metadata['imdb_id'].dropna()).replace('',0).astype(np.int64)

# Rename Meta Data IMDB ID to imdbid
metadata = metadata[metadata.imdb_id.notnull()]
metadata['imdb_id'] = metadata_imdb
metadata = metadata.rename(columns={"imdb_id": "imdbId"})


average_rating = data.groupby('movieId')['rating'].mean().sort_values(ascending = True)
ratings = pd.merge(link, average_rating, on = 'movieId')
print("Ratings dataset is",ratings.shape)


# Let us merge the metadata with the ratings dataset (containing the found average ratings) we just created
merged_data = pd.merge(metadata,ratings, on='imdbId').drop(columns=['adult','poster_path']).drop(columns='belongs_to_collection')
merged_data['id'] = merged_data['id'].astype(np.int64)

# Let us merge the already merged data with the credits dataset
merged_data = pd.merge(merged_data, credit, on = 'id')
print("Final Dataset Size", merged_data.shape)

# lets trying cleaning up the cast name with a function that takes in a dataset, we have chosen the first 5 actors/actresses only
def get_cast_name(dataset):
    name = ""
    if len(ast.literal_eval(dataset)) > 5:
        a = 5
    else:
        a = len(ast.literal_eval(dataset))
    for i in range(a):
     #   name += ast.literal_eval(dataset[0])[i]['character']
     #   name += ", "
        name += ast.literal_eval(dataset)[i]['name']
        name += ", "
    return name

# Replacing the Cast Column with our newly found cast
a = merged_data.copy()
cast_list = []
for i in range(len(merged_data)):
    cast_list.append(get_cast_name(a['cast'][i]))

merged_data['cast'] = cast_list
print("Cast list is done...")

# Renaming more columns
# merged_data.rename(columns={"original_language": "Language", "original_title":"Title"})


# Renaming the genre list
def get_genre(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

genre_list = []
for i in range(len(merged_data)):
    genre_list.append(get_genre(merged_data['genres'][i]))
    
merged_data['genres'] = genre_list
print("Genre List is done...")

# Renaming the country
def get_country(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

country_list = []
for i in range(len(merged_data)):
    country_list.append(get_country(merged_data['production_countries'][i]))
    
merged_data['production_countries'] = country_list
print("Country List is done...")

# Renaming the companies
def get_companies(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

company_list = []
for i in range(len(merged_data)):
    company_list.append(get_country(merged_data['production_companies'][i]))
    
merged_data['production_companies'] = company_list
print("Company list is done...")

# Renaming the Language Column
def get_language(dataset):
    name = ""
    for i in range(len(ast.literal_eval(dataset))):
        name += ast.literal_eval(dataset)[i]['name']
        name += ', '
    return name

lang_list = []
for i in range(len(merged_data)):
    lang_list.append(get_language(merged_data['spoken_languages'][i]))
    
merged_data['spoken_languages'] = lang_list
print("Language list is done...")

# Filling Nan values of the columns
merged_data['homepage'] = merged_data['homepage'].fillna('None')
merged_data['overview'] = merged_data['overview'].fillna('None')
merged_data['release_date'] = merged_data['release_date'].fillna('None')

merged_data = merged_data.rename(columns={"original_language": "Language", "budget":"Budget", "genres":"Genres","homepage":"Webpage","overview":"Overview","production_companies":"Production Company","production_countries":"Country","spoken_languages":"Available Languages","release_date":"Date of Release","title":"Title","cast":"Cast","crew":"Crew","rating":"Rating"})
merged_data = merged_data.drop(columns='original_title')
year = []
for i in range(merged_data.shape[0]):
    year.append(merged_data['Date of Release'][i][:4])
merged_data['Date of Release'] = year

merged_data.head()

Movies dataset is (100004, 3)
Links dataset is (9125, 2)
Keywords dataset is (46419, 2)
Metadata dataset is (45466, 19)
Credits dataset is (45476, 3)
Ratings dataset is (9066, 3)
Final Dataset Size (9031, 20)
Cast list is done...
Genre List is done...
Country List is done...
Company list is done...
Language list is done...


,Budget,Genres,Webpage,id,imdbId,Language,Overview,popularity,Production Company,Country,Date of Release,revenue,runtime,Available Languages,Title,movieId,Rating,Cast,Crew
0,30000000,"Animation, Comedy, Family,",http://toystory.disney.com/toy-story,862,114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,"Pixar Animation Studios,","United States of America,",1995,373554033.0,81.0,"English,",Toy Story,1,3.872470,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"Adventure, Fantasy, Family,",None,8844,113497,en,When siblings Judy and Peter discover an encha...,17.0155,"TriStar Pictures, Teitler Film, Interscope Com...","United States of America,",1995,262797249.0,104.0,"English, Français,",Jumanji,2,3.401869,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"Romance, Comedy,",None,15602,113228,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate,","United States of America,",1995,0.0,101.0,"English,",Grumpier Old Men,3,3.161017,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,16000000,"Comedy, Drama, Romance,",None,31357,114885,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,"Twentieth Century Fox Film Corporation,","United States of America,",1995,81452156.0,127.0,"English,",Waiting to Exhale,4,2.384615,"Whitney Houston, Angela Bassett, Loretta Devin...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,"Comedy,",None,11862,113041,en,Just when George Banks has recovered from his ...,8.38752,"Sandollar Productions, Touchstone Pictures,","United States of America,",1995,76578911.0,106.0,"English,",Father of the Bride Part II,5,3.267857,"Steve Martin, Diane Keaton, Martin Short, Kimb...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


# Movie Recommender (Content Based Filtering)

We will first try content based filtering

In [5]:
# Function to merge relevant columns
def merging_features(data):
    features = []
    for i in range(0, data.shape[0]):
        features.append(data['Genres'][i] + data['Language'][i] + ", " + data['Production Company'][i] + " "+ data['Date of Release'][i] + " " + data['Country'][i]  + " "+data['Title'][i] + " "+data['Cast'][i] )
    return features

In [6]:
merged_data['Merged Features'] = merging_features(merged_data)
merged_data[['Merged Features']].head()

,Merged Features
0,"Animation, Comedy, Family, en, Pixar Animation..."
1,"Adventure, Fantasy, Family, en, TriStar Pictur..."
2,"Romance, Comedy, en, Warner Bros., Lancaster G..."
3,"Comedy, Drama, Romance, en, Twentieth Century ..."
4,"Comedy, en, Sandollar Productions, Touchstone ..."


In [7]:
# Try to find the count of unique words
countmatrix = CountVectorizer().fit_transform(merged_data['Merged Features'])
print(countmatrix.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
# Obtain the angle between the vectorized words
cosinesimilarity = cosine_similarity(countmatrix)
print(cosinesimilarity)

[[1.         0.26923077 0.27456259 ... 0.18156826 0.25064021 0.1132277 ]
 [0.26923077 1.         0.23533936 ... 0.18156826 0.25064021 0.1132277 ]
 [0.27456259 0.23533936 1.         ... 0.18516402 0.25560386 0.11547005]
 ...
 [0.18156826 0.18156826 0.18516402 ... 1.         0.30988989 0.08908708]
 [0.25064021 0.25064021 0.25560386 ... 0.30988989 1.         0.10540926]
 [0.1132277  0.1132277  0.11547005 ... 0.08908708 0.10540926 1.        ]]


In [9]:
# TAKE NOTE THERE ARE TWO THE AVENGERS
merged_data['Index'] = merged_data.index
movie_id = merged_data[merged_data['Title'] == 'Toy Story 2']['Index'].values[0]
movie_id

2505

In [10]:
# Obtain the scores from the cosine similarity to the movie The Avengers
scores = list(enumerate(cosinesimilarity[movie_id]))
sorted_scores = sorted(scores, key = lambda x: x[1], reverse = True)
sorted_scores = sorted_scores[1:]
sorted_scores

[(0, 0.8076923076923074),
 (7508, 0.7647748927656914),
 (8385, 0.6827887419989188),
 (9019, 0.6076436202501997),
 (8027, 0.5883484054145522),
 (8030, 0.5883484054145522),
 (6290, 0.5765566601970552),
 (8043, 0.570781792985393),
 (8051, 0.5547001962252291),
 (7336, 0.5399055247990169),
 (8029, 0.5399055247990169),
 (7671, 0.5262348115842175),
 (8384, 0.5084751798731267),
 (7194, 0.4999999999999999),
 (7802, 0.4907164920657184),
 (7709, 0.48181205582971576),
 (7723, 0.4803844614152615),
 (8671, 0.47343207647399926),
 (3802, 0.4734320764739992),
 (6592, 0.4734320764739992),
 (4599, 0.4706787243316417),
 (6940, 0.4654746681256314),
 (8830, 0.4654746681256314),
 (1867, 0.45790546988962544),
 (6399, 0.45790546988962544),
 (7787, 0.45790546988962544),
 (8020, 0.45790546988962544),
 (7051, 0.4529108136578384),
 (8997, 0.4506939094329988),
 (8290, 0.4447495899966607),
 (6247, 0.4372373160976032),
 (8126, 0.43701422443753773),
 (8539, 0.43701422443753773),
 (6633, 0.4309458036856673),
 (5604, 0.

In [11]:
# Create a loop to print the first 7 Similar Movies
j = 0
print("The top 7 recommended movies to The Avengers" 'are:\n')
for item in sorted_scores:
    movie_title = merged_data[merged_data.Index == item[0]]['Title'].values[0]
    print(j+1, movie_title)
    j+=1
    if(j >= 7):
        break

The top 7 recommended movies to The Avengersare:

1 Toy Story
2 Toy Story 3
3 Toy Story of Terror!
4 Piper
5 Presto
6 One Man Band
7 Luxo Jr.


# TFID Vectorizer Content Based

In [12]:
# We need the title, id, overview column
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9031 entries, 0 to 9030
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Budget               9031 non-null   object 
 1   Genres               9031 non-null   object 
 2   Webpage              9031 non-null   object 
 3   id                   9031 non-null   int64  
 4   imdbId               9031 non-null   int64  
 5   Language             9031 non-null   object 
 6   Overview             9031 non-null   object 
 7   popularity           9031 non-null   object 
 8   Production Company   9031 non-null   object 
 9   Country              9031 non-null   object 
 10  Date of Release      9031 non-null   object 
 11  revenue              9031 non-null   float64
 12  runtime              9031 non-null   float64
 13  Available Languages  9031 non-null   object 
 14  Title                9031 non-null   object 
 15  movieId              9031 non-null   i

In [13]:
# We are converting the Overview portion into a vector
merged_data.head(1)['Overview']

0    Led by Woody, Andy's toys live happily in his ...
Name: Overview, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_copy = merged_data.copy()
tfv = TfidfVectorizer(min_df = 3, max_features=None,
        strip_accents= 'unicode',analyzer='word',
        token_pattern = r'\w{1,}',
        ngram_range=(1,3),
        stop_words = 'english')
# stop_words = 'english', removes , . '' ! and words like this, is,she,is,that which are very repetitive
tfid_copy['Overview'] = tfid_copy['Overview'].fillna('')

In [15]:
# Fitting the Term-Frequency Inverse DF on the overview text
tfv_matrix = tfv.fit_transform(tfid_copy['Overview'])

In [30]:
tfv_matrix.toarray()


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

A sparse matrix is basically a matrix with alot of zeros. The matrix are also the vectors

In [16]:
# Sigmoid is responsible for transforming your values from 0 to 1
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)


In [31]:
sig

array([[0.76161694, 0.76159469, 0.76159427, ..., 0.76159439, 0.76159425,
        0.76159437],
       [0.76159469, 0.76161694, 0.76159533, ..., 0.76159443, 0.76159424,
        0.76159421],
       [0.76159427, 0.76159533, 0.76161694, ..., 0.76159416, 0.7615942 ,
        0.76159419],
       ...,
       [0.76159439, 0.76159443, 0.76159416, ..., 0.76161694, 0.76159441,
        0.76159508],
       [0.76159425, 0.76159424, 0.7615942 , ..., 0.76159441, 0.76161694,
        0.76159418],
       [0.76159437, 0.76159421, 0.76159419, ..., 0.76159508, 0.76159418,
        0.76161694]])

In [17]:
# Reverse mapping of indices and movie_titles
indices = pd.Series(tfid_copy.index, index=tfid_copy['Title']).drop_duplicates()

In [18]:
indices

Title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Rustom                                                9026
Mohenjo Daro                                          9027
The Beatles: Eight Days a Week - The Touring Years    9028
Band of Brothers                                      9029
Cosmos                                                9030
Length: 9031, dtype: int64

In [19]:
indices['Toy Story']
sorted(list(enumerate(sig[indices['Toy Story']])), key=lambda x:x[1], reverse=True)


[(0, 0.7616169400805992),
 (7508, 0.761606537184093),
 (2505, 0.7616045002095143),
 (6173, 0.7615997317294558),
 (437, 0.7615994024837219),
 (6605, 0.7615977518347714),
 (4979, 0.7615974840582976),
 (894, 0.7615971592983352),
 (1603, 0.7615970630459467),
 (6533, 0.7615970183171367),
 (5129, 0.7615964498428696),
 (7693, 0.7615964474362553),
 (402, 0.76159643172821),
 (6089, 0.7615963719806056),
 (2550, 0.7615962927464899),
 (2618, 0.7615961946200176),
 (2693, 0.7615961479668931),
 (6269, 0.7615960477906294),
 (1796, 0.7615960063562671),
 (2773, 0.7615959482396248),
 (8304, 0.7615959400187765),
 (4926, 0.7615959058424667),
 (6199, 0.7615959028322115),
 (4697, 0.7615958990381282),
 (2241, 0.7615958822083184),
 (7504, 0.7615958477993083),
 (1942, 0.7615958362857813),
 (639, 0.7615957855089235),
 (2419, 0.7615957763183933),
 (8395, 0.7615957577356368),
 (4382, 0.7615957527526745),
 (1556, 0.7615957410369059),
 (7345, 0.7615957100259498),
 (5855, 0.7615957057104932),
 (2426, 0.76159567332209

In [20]:
def give_rec(title, sig=sig):
    #Get the index of the movie Title
    idx = indices[title]
    
    sig_scores = sorted(list(enumerate(sig[idx])), key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return tfid_copy['Title'].iloc[movie_indices]

In [21]:
give_rec(title='Star Wars')

954           The Empire Strikes Back
8713     Star Wars: The Force Awakens
967                Return of the Jedi
5078              The Thief of Bagdad
4808                Where Eagles Dare
5166                  The Magic Flute
9022    Kingsglaive: Final Fantasy XV
955                The Princess Bride
279                 The Swan Princess
1657                  Sleeping Beauty
Name: Title, dtype: object

# KNN Collaborative Based Filtering

In [ ]:
# Let us import the datasets, we will be using the smaller datasets first
movies = pd.read_csv('ratings_small.csv').drop(columns = ['timestamp'])
link = pd.read_csv('links_small.csv').drop(columns= ['tmdbId'])
keywords = pd.read_csv('keywords.csv')
metadata = pd.read_csv('movies_metadata.csv').drop(columns=['vote_count', 'vote_average','video','tagline','status'])
credit = pd.read_csv('credits.csv')

print("Movies dataset is", movies.shape)
print("Links dataset is", link.shape)
print("Keywords dataset is", keywords.shape)
print("Metadata dataset is", metadata.shape)
print("Credits dataset is", credit.shape)

movies.head()

In [53]:
# Getting the dataset with each individual UserId and the movies he has watched
merged_data[['Title','imdbId']]
movie_rating = pd.merge(movies,pd.merge(link,merged_data[['Title','imdbId']], on='imdbId' ),on='movieId')
movie_rating.head()

,userId,movieId,rating,imdbId,Title
0,1,31,2.5,112792,Dangerous Minds
1,7,31,3.0,112792,Dangerous Minds
2,31,31,4.0,112792,Dangerous Minds
3,32,31,4.0,112792,Dangerous Minds
4,36,31,3.0,112792,Dangerous Minds


In [77]:
movie_ratingCount = (movie_rating.groupby(by = ['Title'])['rating'].count().reset_index().rename(columns={"rating":"totalRatingCount"}))
movie_ratingCount = pd.merge(movie_rating, movie_ratingCount, on = 'Title')
movie_ratingCount = movie_ratingCount.sort_values(by=['movieId'])
movie_ratingCount.head()

,userId,movieId,rating,imdbId,Title,totalRatingCount
31834,654,1,5.0,114709,Toy Story,247
31681,242,1,5.0,114709,Toy Story,247
31680,241,1,3.0,114709,Toy Story,247
31679,240,1,4.5,114709,Toy Story,247
31678,237,1,3.0,114709,Toy Story,247


Setting a popularity threshold, where we query only movies that meet the threshold condition

In [78]:
popularity_threshold = 40
rating_popular = movie_ratingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular.head()

,userId,movieId,rating,imdbId,Title,totalRatingCount
31834,654,1,5.0,114709,Toy Story,247
31681,242,1,5.0,114709,Toy Story,247
31680,241,1,3.0,114709,Toy Story,247
31679,240,1,4.5,114709,Toy Story,247
31678,237,1,3.0,114709,Toy Story,247


In [80]:
rating_popular.shape

(51097, 6)

In [82]:
# Lets create a Pivot Matrix
movie_features_df = rating_popular.pivot_table(index='Title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
Title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28 Days Later,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df)

# We will be using K-Nearest Neighbours 
from sklearn.neighbors import NearestNeighbors

# Once again we will be using cosine similarity
model_knn = NearestNeighbors(metric ='cosine', algorithm ='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [87]:
# Shape of the pivot table
movie_features_df.shape

(625, 671)

In [109]:
# Query index picks a random movie among the 625 entries of movies
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)

# Choose the 6 nearest neighbours. Reshape is because we receive a Series
distance, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1)
                                       , n_neighbors=6)

353


Distance is based on Cosine Similarity of K-Nearest Neighbors with Collarborative Filtering

In [110]:
for i in range(0, len(distance.flatten())):
    if i == 0:
        print("Recommendation for {0}:\n".format(movie_features_df.index[query_index]))
    else:
        print("{0}: {1} with distance of {2}:".format(i, movie_features_df.index[indices.flatten()[i]],distance.flatten()[i]))

Recommendation for Predator:

1: RoboCop with distance of 0.3932312922896629:
2: Alien with distance of 0.45328814215940694:
3: Alien³ with distance of 0.46617317511788436:
4: Total Recall with distance of 0.49765589562119517:
5: The Terminator with distance of 0.5163099459236367:


# Golden Village

In [ ]:
# Singular Value Decomposition (SVD) Netflix way Collaborative Filtering
# Can we scrap Golden Village Data and recommend a on going movie 

In [177]:
# Importing the Golden Village dataset and filling nan values
gv = pd.read_csv('GVmovie.csv')
gv['Subtitle'] = gv['Subtitle'].fillna('None')
gv['Cast'] = gv['Cast'].fillna('None')
gv['runtime'] = gv['runtime'].fillna(0)
gv['Index'] = gv.index
gv.head()

,Title,Genres,Language,Subtitle,Date of Release,runtime,Cast,Director,Distributor,Overview,Index
0,The Marksman,Action/ Thriller,English,Chinese,25/2/2021,107.0,"Liam Neeson, Katheryn Winnick, Teresa Ruiz, Ju...",Robert Lorenz,GVP,Hardened Arizona rancher Jim Hanson (Liam Nees...,0
1,The Diam Diam Era Two 我们的故事之沉默的年代2,Drama/ Comedy,Mandarin,"English, Chinese",11/2/2021,91.0,"Mark Lee 李国煌, Richie Koh 许瑞奇, Danny Lee 李博 翔, ...",Jack Neo 梁志强,GVP,"The story continues with Ah Kun, Osman and Sha...",1
2,Tom & Jerry,Animation/ Comedy/ Family,English,Chinese,11/2/2021,101.0,"Chloe Grace Moretz, Michael Peña, Colin Jost, ...",Tim Story,WB,One of the most beloved rivalries in history i...,2
3,Doraemon The Movie: Nobita's New Dinosaur,Animation / Adventure,Japanese,"English, Chinese",25/2/2021,110.0,"Megumi Oohara, Wasabi Mizuta, Yumi Kakazu, Sub...",Kazuaki Imai,Odex,"Nobita hatches two new dinosaurs, named Kyu an...",3
4,New Gods: Nezha Reborn 新神榜：哪吒重生,Action/ Animation/ Fantasy,Mandarin,"English, Chinese",18/2/2021,117.0,"Yang Tianxiang 杨天翔, Zhang He 张赫, Xuan Xiaoming...",Ji Zhao 赵霁,GVP,The story happens after over 2000 years when N...,4


In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer
gv_copy = gv.copy()
tfv = TfidfVectorizer(min_df = 3, max_features=None,
        strip_accents= 'unicode',analyzer='word',
        token_pattern = r'\w{1,}',
        ngram_range=(1,3),
        stop_words = 'english')

tfv_matrix = tfv.fit_transform(gv['Overview'])

In [71]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel and cosine similarity
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
cosinesimilarity = cosine_similarity(tfv_matrix)

In [87]:
index = gv[gv.Title == 'Tom & Jerry']['Index'].values[0]
sigmoid_rec = sorted(list(enumerate(sig[index])), key=lambda x:x[1], reverse=True)
cosine_rec = sorted(list(enumerate(cosinesimilarity[index])), key=lambda x:x[1], reverse=True)

In [100]:
count = 1
print("Golden Village movies to consider(Cosine Similarity):")
for i in cosine_rec[1:6]:
    print(count,":",gv.iloc[i[0]]['Title'])
    count+=1
count = 1
print("\n")
print("Golden Village movies to consider(Sigmoid Kernel):")
for i in sigmoid_rec[1:6]:
    print(count,":",gv.iloc[i[0]]['Title'])
    count+=1

Golden Village movies to consider(Cosine Similarity):
1 : Promising Young Woman
2 : Chaos Walking
3 : Wonder Woman 1984
4 : André And His Olive Tree
5 : New Year Blues


Golden Village movies to consider(Sigmoid Kernel):
1 : Promising Young Woman
2 : Chaos Walking
3 : Wonder Woman 1984
4 : André And His Olive Tree
5 : New Year Blues


##### We will now attempt to merge our TFID Vectorizer to recommend a Golden Village Movie

In [126]:
# This is the code for our content based recommender using TFID Vectorizer with a Sigmoid Kernel
# merged_data.copy() is using our processed merged_data from the one line code processing part
tfid_copy = merged_data.copy()
tfv = TfidfVectorizer(min_df = 3, max_features=None,
        strip_accents= 'unicode',analyzer='word',
        token_pattern = r'\w{1,}',
        ngram_range=(1,3),
        stop_words = 'english')
# stop_words = 'english', removes , . '' ! and words like this, is,she,is,that which are very repetitive
tfid_copy['Overview'] = tfid_copy['Overview'].fillna('')

tfv_matrix = tfv.fit_transform(tfid_copy['Overview'])


# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(tfid_copy.index, index=tfid_copy['Title']).drop_duplicates()

indices['Toy Story']
sorted(list(enumerate(sig[indices['Toy Story']])), key=lambda x:x[1], reverse=True)

def give_rec(title, sig=sig):
    #Get the index of the movie Title
    idx = indices[title]
    
    sig_scores = sorted(list(enumerate(sig[idx])), key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return tfid_copy.iloc[movie_indices]


In [213]:
selected = pd.DataFrame(give_rec(title='Toy Story')['Overview']).iloc[0]

gv_merged = pd.DataFrame(gv_copy['Overview'].append(selected)).rename(columns={0:"Overview"})

In [214]:
length = [i for i in range(len(gv_merged))]
gv_merged = gv_merged.set_index(pd.Index(length))
gv_merged

,Overview
0,Hardened Arizona rancher Jim Hanson (Liam Nees...
1,"The story continues with Ah Kun, Osman and Sha..."
2,One of the most beloved rivalries in history i...
3,"Nobita hatches two new dinosaurs, named Kyu an..."
4,The story happens after over 2000 years when N...
5,Chen Xiaomeng is a theatre actor who just can’...
6,A WRITER’S ODYSSEY tells the story of Kongwen ...
7,A tender and sweeping story about what roots u...
8,Following the excitement of first Bangkok and ...
9,From visionary director Emerald Fennell (Killi...


In [215]:
tfv_matrix = tfv.fit_transform(gv_merged['Overview'])
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)
cosinesimilarity = cosine_similarity(tfv_matrix)
index = 47 # 47 is the first movie from our TFID Content Based, 
sigmoid_rec = sorted(list(enumerate(sig[index])), key=lambda x:x[1], reverse=True)
cosine_rec = sorted(list(enumerate(cosinesimilarity[index])), key=lambda x:x[1], reverse=True)

In [216]:
count = 1
print("Golden Village movies to consider(Sigmoid Kernel):")
for i in sigmoid_rec[1:6]:
    if(i[1] == 0):
        print("No movies to recommend!")
        break
    print(count,":",gv.iloc[i[0]]['Title'])
    count+=1

Golden Village movies to consider(Sigmoid Kernel):
1 : André And His Olive Tree
2 : 76 Days
3 : Fatherhood
4 : Nobody
5 : Minari
